# How to run

You can run each cell in order with shift+enter, or run the whole thing at once by using the Kernel menu up above and hitting 'Restart & Run All'

This notebook expects a folder structure as follows
```
---
 | powers.ipynb
 |-data\Orcus - Powers.xlsx
 |-output
```

and will take data\Orcus - Powers.xlsx and create a markdown file output\powers.md

We use Pandas to read in the excel spreadsheet, and define a function which takes a row in the Pandas DataFrame and puts all the values in the right place (with a little logic to handle fields which aren't always there) to create a markdown formatted string.

It's pretty similar to what the existing Mail Merge does, but we have complete control over how we build the string and map values. The key Pythonic bit is the curly braces {} and the `**dict`. Curly braces indicate a place that we want to input a value in a string with a **key**, and ** takes a dictionary and puts the values with the proper keys in the proper place.  

```
'{hello}'.format(**{'hello':'world', 'not_needed':'mars'})
evaluates to
'world'
```


Then we join that list of strings and save a file.  For more on Python, I recommend [Automate the Boring Stuff with Python](https://automatetheboringstuff.com/)

In [1]:
# cell 0, standard imports

import pandas as pd
import os

dot  = '●'

In [2]:
#cell 1, read in data from filename

filename = 'Orcus - Powers.xlsx'

#by default Pandas loads missing values with ugly NaNs, so we replace those with the empty string ''
data = pd.read_excel(os.path.join('data', filename)).fillna('')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data\\Orcus - Powers.xlsx'

In [ ]:
#cell 2, examine 5 random powers

data.sample(5)

In [ ]:
#cell 3
#we define a function which takes in a row of this power dataframe and returns an md string

def power_to_md(row):
    
    #converting the row to a dictionary let's use do some cool stuff with placing values in strings using .format(**row_dict)
    row_dict = dict(row)
    
    #remove leading and trailing whitespace
    for k, v in row_dict.items():
        row_dict[k] = str(v).strip()
    
    # we use completion to check a few edge cases
    completion = dict()
    for key,value in row.items():
        if value:
            completion[key] = value
    
    # we check if only Name is filled in, and return a ## Header line
    if list(completion.keys()) == ['Name']:
        md = '## {Name}'.format(**row_dict)
        return md
    
    # we check if nothing is filling in and return an empty row
    if list(completion.keys()) == []:
        md = ''
        return md
    
    #start with name. md is the string we'll be outputing
    md = "#### {Name}".format(**row_dict)
    
    #add flavor if it exists
    #'\n' is Python for new line
    if row['Flavor']:
        md += "\n*{Flavor}*".format(**row_dict)
        
    #modify tags a little, since they aren't always present    
    if row_dict.get('Tags'):
        row_dict['Tags'] = '● **{Tags}**'.format(**row_dict)
    
    #this is a complex chunk
    md += """\n**{Frequency}** **{Action}**
**{List}** **{Category}** **{Tier}** {Tags}
**{Range}** {Range Details}""".format(**row_dict)
    
    #if a field exists, we add it
    if row['Requirement']:
        md += '\n**Requirements** {Requirement}'.format(**row_dict)
        
    if row['Attack']:
        md += '\n**Attack** {Attack} vs {Defence}'.format(**row_dict)
        
    if row['Hit']:
        md += '\n**Hit** {Hit}'.format(**row_dict)
    
    if row['Miss']:
        md += '\n**Miss** {Miss}'.format(**row_dict)
        
    if row['Effect']:
        md += '\n**Effect** {Effect}'.format(**row_dict)
        
    if row['Special']:
        md += '\n**Special** {Special}'.format(**row_dict)
        
    if row['Maintain']:
        md += '\n**Maintain** {Maintain}'.format(**row_dict)
        
    if row['Boost']:
        md += '\n**Boost** {Boost}'.format(**row_dict)
    
    #finally we remove the empty tags
    return md.replace('****','')


In [ ]:
#cell 4
#this cell converts the dataframe into a list of markdown formatted strings using the function defined above

output = []

for idx, row in data.iterrows():
    md = power_to_md(row)
    if md: #this removes empty lines, which you may not want
        output.append(md)

In [ ]:
#cell 5
#we can look at elements here using Python list slicing

for elem in output[100:110]:
    print(elem)
    print('')

In [ ]:
#cell 6
# now we have to save to disk

fname =  'Orcus Powers.md'

with open(os.path.join('output', fname), 'w', encoding='utf-8') as f:
    f.write('\n\n'.join(output))